In [ ]:
import polars as pl
import pandas as pd
import numpy as np

<module 'polars' from '/Users/egortuzarov/Library/Caches/pypoetry/virtualenvs/homecredit-_roIwX6z-py3.11/lib/python3.11/site-packages/polars/__init__.py'>